In [1]:
%pip install opencv-python
%pip install numpy
%pip install matplotlib
%pip install ultralytics
%pip install ipywidgets
%pip install boto3

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from concurrent.futures import ThreadPoolExecutor
from ultralytics import YOLO

# load the yolov8 model for object detection
model = YOLO('yolov8n.pt')

def process_frame(frame, width, height, person_tracks, vehicle_tracks):
    # run object detection on the frame
    results = model.track(frame, persist=True, tracker="bytetrack.yaml")
    if results[0].boxes is not None:
        # get bounding boxes, class ids, and track ids
        boxes = results[0].boxes.xyxy.cpu().numpy()
        class_ids = results[0].boxes.cls.cpu().numpy()
        track_ids = results[0].boxes.id.cpu().numpy() if hasattr(results[0].boxes, 'id') else np.array([])

        # loop through detected objects
        for box, class_id, track_id in zip(boxes, class_ids, track_ids):
            # calculate center of the bounding box
            center_x = (box[0] + box[2]) / 2 / width
            center_y = (box[1] + box[3]) / 2 / height

            # draw the bounding box and label
            x1, y1, x2, y2 = map(int, box)
            label = f"ID {int(track_id)}"
            color = (0, 255, 0) if class_id == 0 else (0, 0, 255)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # save the tracking points for each class
            if class_id == 0:
                person_tracks.setdefault(track_id, []).append((center_x, center_y))
            elif class_id in [2, 3, 5, 7]:
                vehicle_tracks.setdefault(track_id, []).append((center_x, center_y))
    return frame

def process_video_and_generate_output(video_path, output_video_path, animation_output_path, final_graph_path):
    # open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise FileNotFoundError(f"Could not open video file: {video_path}")

    # get video properties like width, height, and fps
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # initialize the video writer for saving the output video
    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))
    person_tracks, vehicle_tracks = {}, {}

    # use thread pool to process frames concurrently
    with ThreadPoolExecutor() as executor:
        future_frames = []
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            future = executor.submit(process_frame, frame, width, height, person_tracks, vehicle_tracks)
            future_frames.append(future)

        # write the processed frames to the output video
        for future in future_frames:
            frame = future.result()
            out.write(frame)

    cap.release()
    out.release()

    # create an animated plot of the tracks
    fig, ax = plt.subplots(figsize=(8, 8))
    def update(frame_idx):
        ax.clear()
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.grid(True)
        ax.set_aspect('equal', adjustable='box')

        # plot tracks up to the current frame
        for points in person_tracks.values():
            if frame_idx < len(points):
                points_arr = np.array(points)
                ax.plot(points_arr[:frame_idx + 1, 0], points_arr[:frame_idx + 1, 1], 'b-', alpha=0.7)
        for points in vehicle_tracks.values():
            if frame_idx < len(points):
                points_arr = np.array(points)
                ax.plot(points_arr[:frame_idx + 1, 0], points_arr[:frame_idx + 1, 1], 'r-', alpha=0.7)

    # save the animation as a gif
    anim = FuncAnimation(fig, update, frames=max(len(points) for points in list(person_tracks.values()) + list(vehicle_tracks.values())), interval=1000 // fps)
    anim.save(animation_output_path, writer=PillowWriter(fps=fps))
    plt.close(fig)

    # plot the final tracks and save as an image
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.grid(True)
    ax.set_aspect('equal', adjustable='box')

    # plot the complete tracks for persons and vehicles
    for points in person_tracks.values():
        points_arr = np.array(points)
        ax.plot(points_arr[:, 0], points_arr[:, 1], 'b-', alpha=0.7)
    for points in vehicle_tracks.values():
        points_arr = np.array(points)
        ax.plot(points_arr[:, 0], points_arr[:, 1], 'r-', alpha=0.7)

    # save the final graph
    plt.savefig(final_graph_path)
    plt.close(fig)

# example usage with paths to input and output files
video_path = r"C:\Users\aarus\Downloads\example3.mp4"
output_video_path = r"C:\Users\aarus\Downloads\example3output.mp4"
animation_output_path = r"C:\Users\aarus\Downloads\example3gif.gif"
final_graph_path = r"C:\Users\aarus\Downloads\example3graph.jpg"

process_video_and_generate_output(video_path, output_video_path, animation_output_path, final_graph_path)



Ultralytics 8.3.59  Python-3.11.9 torch-2.5.1+cpu CPU (13th Gen Intel Core(TM) i9-13900H)

Ultralytics 8.3.59  Python-3.11.9 torch-2.5.1+cpu CPU (13th Gen Intel Core(TM) i9-13900H)

Ultralytics 8.3.59  Python-3.11.9 torch-2.5.1+cpu CPU (13th Gen Intel Core(TM) i9-13900H)

Ultralytics 8.3.59  Python-3.11.9 torch-2.5.1+cpu CPU (13th Gen Intel Core(TM) i9-13900H)

Ultralytics 8.3.59  Python-3.11.9 torch-2.5.1+cpu CPU (13th Gen Intel Core(TM) i9-13900H)

Ultralytics 8.3.59  Python-3.11.9 torch-2.5.1+cpu CPU (13th Gen Intel Core(TM) i9-13900H)

Ultralytics 8.3.59  Python-3.11.9 torch-2.5.1+cpu CPU (13th Gen Intel Core(TM) i9-13900H)

Ultralytics 8.3.59  Python-3.11.9 torch-2.5.1+cpu CPU (13th Gen Intel Core(TM) i9-13900H)

Ultralytics 8.3.59  Python-3.11.9 torch-2.5.1+cpu CPU (13th Gen Intel Core(TM) i9-13900H)


Ultralytics 8.3.59  Python-3.11.9 torch-2.5.1+cpu CPU (13th Gen Intel Core(TM) i9-13900H)
Ultralytics 8.3.59  Python-3.11.9 torch-2.5.1+cpu CPU (13th Gen Intel Core(TM) i9-13900H)